In [ ]:
import pandas as pd

df = pd.read_csv('/content/Task sheet - control tower - Raw data.csv')

current_time = pd.Timestamp('2026-01-08 09:00:00')

date_cols = ['Order Created At', 'Shipment picked up At', 'Out For Delivery At', 'Delivered At']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [ ]:
df.describe()

,Order Created At,Shipment picked up At,Out For Delivery At,Delivered At,Destination Pincode,Promised SLA (Hours),Shipment Weight (Kg),Order Value (INR)
count,5000,4314,3897,3198,5000.000000,5000.000000,5000.000000,5000.000000
mean,2025-12-29 08:17:25.080600064,2025-12-29 12:11:23.358136064,2025-12-30 14:21:03.158840320,2025-12-30 20:06:04.859599872,425420.924800,37.348800,1.076698,786.685400
min,2025-12-22 08:14:14,2025-12-22 08:29:53,2025-12-22 22:24:32,2025-12-23 00:24:32,110001.000000,-24.000000,0.100000,99.000000
25%,2025-12-25 20:53:51,2025-12-26 00:40:05.249999872,2025-12-27 01:42:51,2025-12-27 07:58:46.249999872,400050.000000,24.000000,0.600000,450.000000
50%,2025-12-29 06:12:50.500000,2025-12-29 09:56:04.500000,2025-12-30 11:35:05,2025-12-30 17:43:06,500001.000000,36.000000,0.900000,673.000000
75%,2026-01-01 21:50:06.500000,2026-01-02 02:13:55.500000,2026-01-03 03:21:17,2026-01-03 08:54:06,560068.000000,48.000000,1.340000,994.000000
max,2026-01-05 08:49:05,2026-01-06 02:37:38,2026-01-07 12:09:27,2026-01-07 20:06:11,600096.000000,72.000000,8.550000,5085.000000
std,NaN,NaN,NaN,NaN,166364.773292,13.396083,0.703948,492.271944


In [ ]:
print(df.isnull().sum())
print("\n")

AWB                           0
Order Created At              0
Shipment picked up At       686
Out For Delivery At        1103
Delivered At               1802
Current Shipment Status       0
Courier Partner               0
City                          0
Store ID                      0
Destination Pincode           0
Payment Type                  0
Promised SLA (Hours)          0
Shipment Weight (Kg)          0
Order Value (INR)             0
dtype: int64




In [ ]:
time_travel = df[df['Delivered At'] < df['Shipment picked up At']]
print(f"Time Travel (Delivered < Pickup): {len(time_travel)}")

Time Travel (Delivered < Pickup): 0


In [ ]:
duplicates = df[df.duplicated(subset=['AWB'], keep=False)]
print(f"Duplicate AWBs: {len(duplicates)}")

Duplicate AWBs: 114


In [ ]:
fake_delivered = df[(df['Current Shipment Status'] == 'DELIVERED') & (df['Delivered At'].isna())]
print(f"Status is DELIVERED but no Timestamp: {len(fake_delivered)}")

Status is DELIVERED but no Timestamp: 50


In [ ]:
active_statuses = ['PICKED_UP', 'IN_TRANSIT', 'OUT_FOR_DELIVERY', 'DELIVERED']
ghost_pickups = df[(df['Current Shipment Status'].isin(active_statuses)) & (df['Shipment picked up At'].isna())]
print(f"Active Status but no Pickup Timestamp: {len(ghost_pickups)}")

Active Status but no Pickup Timestamp: 9


In [ ]:
negative_values = df[(df['Order Value (INR)'] < 0) | (df['Shipment Weight (Kg)'] < 0)]
print(f"Rows with Negative Value/Weight: {len(negative_values)}")

Rows with Negative Value/Weight: 0


In [ ]:
not_picked_up = df[df['Shipment picked up At'].isna()]
stuck_in_transit = df[df['Shipment picked up At'].notna() & df['Out For Delivery At'].isna()]
failed_delivery = df[df['Out For Delivery At'].notna() & df['Delivered At'].isna()]
print(f"1. Total Orders Created:  {len(df)}")
print(f"2. Picked Up:             {df['Shipment picked up At'].notna().sum()}")
print(f"3. Out for Delivery:      {df['Out For Delivery At'].notna().sum()}")
print(f"4. Successfully Delivered:{df['Delivered At'].notna().sum()}")
print("\n")


1. Total Orders Created:  5000
2. Picked Up:             4314
3. Out for Delivery:      3897
4. Successfully Delivered:3198




In [ ]:
print(f"\n1. MISSED PICKUPS (Total: {len(not_picked_up)})")
print(not_picked_up['Courier Partner'].value_counts().head(3))
print(f"\n2. STUCK IN TRANSIT (Total: {len(stuck_in_transit)})")
print(stuck_in_transit['Courier Partner'].value_counts().head(3))
print(f"\n3. FAILED DELIVERIES (Total: {len(failed_delivery)})")
print(failed_delivery['Courier Partner'].value_counts().head(3))


1. MISSED PICKUPS (Total: 686)
Courier Partner
CarrierB    190
CarrierA    147
CarrierD    139
Name: count, dtype: int64

2. STUCK IN TRANSIT (Total: 417)
Courier Partner
CarrierB    108
CarrierA    108
CarrierC     80
Name: count, dtype: int64

3. FAILED DELIVERIES (Total: 699)
Courier Partner
CarrierB    178
CarrierA    162
CarrierC    158
Name: count, dtype: int64


In [ ]:

df_clean = df.drop_duplicates(subset=['AWB'], keep='first').copy()

bad_delivered = (df_clean['Current Shipment Status'] == 'DELIVERED') & (df_clean['Delivered At'].isna())
df_clean = df_clean[~bad_delivered]

active_statuses = ['PICKED_UP', 'IN_TRANSIT', 'OUT_FOR_DELIVERY', 'DELIVERED', 'FAILED_DELIVERY']
bad_pickup = (df_clean['Current Shipment Status'].isin(active_statuses)) & (df_clean['Shipment picked up At'].isna())
df_clean = df_clean[~bad_pickup]


time_travel = (df_clean['Delivered At'] < df_clean['Shipment picked up At'])
df_clean = df_clean[~time_travel]

print(f"Original Rows: {len(df)}")
print(f"Cleaned Rows:  {len(df_clean)}")
print(f"Removed:       {len(df) - len(df_clean)} bad records")

Original Rows: 5000
Cleaned Rows:  4889
Removed:       111 bad records


In [ ]:
df_clean.describe()

,Order Created At,Shipment picked up At,Out For Delivery At,Delivered At,Destination Pincode,Promised SLA (Hours),Shipment Weight (Kg),Order Value (INR)
count,4889,4223,3812,3164,4889.000000,4889.000000,4889.000000,4889.000000
mean,2025-12-29 08:36:31.381059584,2025-12-29 12:41:27.456310528,2025-12-30 14:48:32.841553152,2025-12-30 20:24:07.563211008,424864.536715,37.389241,1.075696,784.797096
min,2025-12-22 08:14:14,2025-12-22 08:29:53,2025-12-22 22:24:32,2025-12-23 00:24:32,110001.000000,-24.000000,0.100000,99.000000
25%,2025-12-25 20:59:56,2025-12-26 01:22:18.500000,2025-12-27 02:09:33.249999872,2025-12-27 07:55:42.249999872,400050.000000,24.000000,0.600000,450.000000
50%,2025-12-29 06:48:08,2025-12-29 11:15:49,2025-12-30 12:25:47,2025-12-30 18:04:28.500000,411057.000000,36.000000,0.890000,673.000000
75%,2026-01-01 22:01:12,2026-01-02 02:38:36,2026-01-03 03:54:37,2026-01-03 09:15:15.500000,560068.000000,48.000000,1.340000,989.000000
max,2026-01-05 08:49:05,2026-01-06 02:37:38,2026-01-07 12:09:27,2026-01-07 20:06:11,600096.000000,72.000000,8.550000,5085.000000
std,NaN,NaN,NaN,NaN,166634.228842,13.453109,0.705513,489.812344


In [ ]:
df_clean['is_pickup_breach'] = (df_clean['Current Shipment Status'] == 'PICKUP_PENDING') & \
                               ((current_time - df_clean['Order Created At']).dt.total_seconds() / 3600 > 24)

# Flag B: Transit SLA Breach (Active shipments exceeding SLA)
active_status = ['IN_TRANSIT', 'PICKED_UP']
df_clean['is_transit_breach'] = (df_clean['Current Shipment Status'].isin(active_status)) & \
                                ((current_time - df_clean['Shipment picked up At']).dt.total_seconds() / 3600 > df_clean['Promised SLA (Hours)'])

# Flag C: Failed Delivery
df_clean['is_failed_delivery'] = (df_clean['Current Shipment Status'] == 'FAILED_DELIVERY')


In [ ]:
city_correlation = df_clean.groupby('City').agg(
    Total_Shipments=('AWB', 'count'),
    Pickup_Breaches=('is_pickup_breach', 'sum'),
    Transit_Breaches=('is_transit_breach', 'sum'),
    Failed_Deliveries=('is_failed_delivery', 'sum')
)
city_correlation['Pickup_Fail_Rate_%'] = ((city_correlation['Pickup_Breaches'] / city_correlation['Total_Shipments']) * 100).round(1)
city_correlation['Transit_Fail_Rate_%'] = ((city_correlation['Transit_Breaches'] / city_correlation['Total_Shipments']) * 100).round(1)
city_correlation['Delivery_Fail_Rate_%'] = ((city_correlation['Failed_Deliveries'] / city_correlation['Total_Shipments']) * 100).round(1)

In [ ]:
store_correlation = df_clean[df_clean['is_pickup_breach']].groupby(['City', 'Store ID']).size().reset_index(name='Pending_Count')

In [ ]:
print(city_correlation.sort_values('Transit_Fail_Rate_%', ascending=True))
print(store_correlation.sort_values('Pending_Count', ascending=True).head(5))

           Total_Shipments  Pickup_Breaches  Transit_Breaches  \
City                                                            
Bangalore             1048               48                75   
Mumbai                 882               43                72   
Delhi                  910               34                81   
Pune                   659               28                62   
Hyderabad              677               26                66   
Chennai                713               27                77   

           Failed_Deliveries  Pickup_Fail_Rate_%  Transit_Fail_Rate_%  \
City                                                                    
Bangalore                 51                 4.6                  7.2   
Mumbai                    38                 4.9                  8.2   
Delhi                     43                 3.7                  8.9   
Pune                      45                 4.2                  9.4   
Hyderabad                 27             

In [ ]:
df_clean.corr(numeric_only=True)

,Destination Pincode,Promised SLA (Hours),Shipment Weight (Kg),Order Value (INR),is_pickup_breach,is_transit_breach,is_failed_delivery
Destination Pincode,1.000000,-0.114112,0.029163,0.005683,0.006207,0.001930,-0.000261
Promised SLA (Hours),-0.114112,1.000000,-0.011006,0.027968,-0.004404,-0.020634,0.012127
Shipment Weight (Kg),0.029163,-0.011006,1.000000,0.018119,0.016174,-0.007599,-0.025295
Order Value (INR),0.005683,0.027968,0.018119,1.000000,0.008427,0.023406,-0.008461
is_pickup_breach,0.006207,-0.004404,0.016174,0.008427,1.000000,-0.065380,-0.047549
is_transit_breach,0.001930,-0.020634,-0.007599,0.023406,-0.065380,1.000000,-0.070671
is_failed_delivery,-0.000261,0.012127,-0.025295,-0.008461,-0.047549,-0.070671,1.000000


In [ ]:
df_clean['is_pickup_breach'] = (df_clean['Current Shipment Status'] == 'PICKUP_PENDING') & \
                               ((current_time - df_clean['Order Created At']).dt.total_seconds() / 3600 > 24)
active_status = ['IN_TRANSIT', 'PICKED_UP']
df_clean['is_transit_breach'] = (df_clean['Current Shipment Status'].isin(active_status)) & \
                                ((current_time - df_clean['Shipment picked up At']).dt.total_seconds() / 3600 > df_clean['Promised SLA (Hours)'])
df_clean['is_failed_delivery'] = (df_clean['Current Shipment Status'] == 'FAILED_DELIVERY')

In [ ]:
store_matrix = df_clean.groupby(['Store ID', 'City']).agg(
    Total_Shipments=('AWB', 'count'),
    Pickup_Breaches=('is_pickup_breach', 'sum'),
    Transit_Breaches=('is_transit_breach', 'sum'),
    Failed_Deliveries=('is_failed_delivery', 'sum')
).reset_index()
store_matrix['Pickup_Fail_%'] = ((store_matrix['Pickup_Breaches'] / store_matrix['Total_Shipments']) * 100).round(1)
store_matrix['Transit_Fail_%'] = ((store_matrix['Transit_Breaches'] / store_matrix['Total_Shipments']) * 100).round(1)
store_matrix['Delivery_Fail_%'] = ((store_matrix['Failed_Deliveries'] / store_matrix['Total_Shipments']) * 100).round(1)

In [ ]:
print("--- 🚨 TOP 5 STORES: PICKUP FAILURES ---")
print(store_matrix.sort_values('Pickup_Fail_%', ascending=False).head(5)[['Store ID', 'City', 'Total_Shipments', 'Pickup_Breaches', 'Pickup_Fail_%']])

print("\n--- ⏳ TOP 5 STORES: TRANSIT STUCK ---")
print(store_matrix.sort_values('Transit_Fail_%', ascending=False).head(5)[['Store ID', 'City', 'Total_Shipments', 'Transit_Breaches', 'Transit_Fail_%']])

print("\n--- 📦 TOP 5 STORES: FAILED DELIVERIES ---")
print(store_matrix.sort_values('Delivery_Fail_%', ascending=False).head(5)[['Store ID', 'City', 'Total_Shipments', 'Failed_Deliveries', 'Delivery_Fail_%']])

--- 🚨 TOP 5 STORES: PICKUP FAILURES ---
   Store ID       City  Total_Shipments  Pickup_Breaches  Pickup_Fail_%
80      S14      Delhi               37                4           10.8
87      S15  Hyderabad               39                4           10.3
2       S01      Delhi               61                6            9.8
11      S02       Pune               31                3            9.7
23      S04       Pune               32                3            9.4

--- ⏳ TOP 5 STORES: TRANSIT STUCK ---
   Store ID       City  Total_Shipments  Transit_Breaches  Transit_Fail_%
79      S14    Chennai               48                 8            16.7
8       S02      Delhi               55                 9            16.4
39      S07  Hyderabad               43                 7            16.3
17      S03       Pune               49                 8            16.3
43      S08    Chennai               51                 8            15.7

--- 📦 TOP 5 STORES: FAILED DELIVERIES ---
  

In [ ]:
carrier_city_matrix = pd.crosstab(df_clean['Courier Partner'], df_clean['City'])

# 5. Calculate Distribution Percentages (Row-wise)
# "How is each Courier's load distributed across cities?"
carrier_city_pct = pd.crosstab(df_clean['Courier Partner'], df_clean['City'], normalize='index') * 100

print("--- COURIER vs. CITY VOLUME MATRIX ---")
print(carrier_city_matrix)

print("\n--- COURIER vs. CITY DISTRIBUTION (% of Courier's Total Volume) ---")
print(carrier_city_pct.round(1))

--- COURIER vs. CITY VOLUME MATRIX ---
City             Bangalore  Chennai  Delhi  Hyderabad  Mumbai  Pune
Courier Partner                                                    
CarrierA               239      160    196        156     199   146
CarrierB               300      201    227        175     238   180
CarrierC               222      147    184        143     192   129
CarrierD               157      119    183        121     138   135
CarrierE               130       86    120         82     115    69

--- COURIER vs. CITY DISTRIBUTION (% of Courier's Total Volume) ---
City             Bangalore  Chennai  Delhi  Hyderabad  Mumbai  Pune
Courier Partner                                                    
CarrierA              21.8     14.6   17.9       14.2    18.2  13.3
CarrierB              22.7     15.2   17.2       13.2    18.0  13.6
CarrierC              21.8     14.5   18.1       14.1    18.9  12.7
CarrierD              18.4     14.0   21.5       14.2    16.2  15.8
CarrierE

In [ ]:
current_time = pd.Timestamp('2026-01-08 09:00:00')

# Flag A: Pending Pickup > 24 Hours
df_clean['is_pickup_breach'] = (df_clean['Current Shipment Status'] == 'PICKUP_PENDING') & \
                               ((current_time - df_clean['Order Created At']).dt.total_seconds() / 3600 > 24)

# Flag B: Transit SLA Breach
active_status = ['IN_TRANSIT', 'PICKED_UP']
df_clean['is_transit_breach'] = (df_clean['Current Shipment Status'].isin(active_status)) & \
                                ((current_time - df_clean['Shipment picked up At']).dt.total_seconds() / 3600 > df_clean['Promised SLA (Hours)'])

# Flag C: Failed Delivery
df_clean['is_failed_delivery'] = (df_clean['Current Shipment Status'] == 'FAILED_DELIVERY')

# 5. Aggregate by Store ID
store_stats = df_clean.groupby(['Store ID', 'City']).agg(
    Total_Shipments=('AWB', 'count'),
    Pickup_Breaches=('is_pickup_breach', 'sum'),
    Transit_Breaches=('is_transit_breach', 'sum'),
    Failed_Deliveries=('is_failed_delivery', 'sum')
).reset_index()

# 6. Calculate Percentages
store_stats['Pickup_Fail_%'] = ((store_stats['Pickup_Breaches'] / store_stats['Total_Shipments']) * 100).round(1)
store_stats['Transit_Fail_%'] = ((store_stats['Transit_Breaches'] / store_stats['Total_Shipments']) * 100).round(1)
store_stats['Delivery_Fail_%'] = ((store_stats['Failed_Deliveries'] / store_stats['Total_Shipments']) * 100).round(1)

# 7. Print Top 5 Tables
print("--- 1. TOP 5 STORES: PICKUP FAILURE % ---")
print(store_stats.sort_values('Pickup_Fail_%', ascending=False).head(5)[['Store ID', 'City', 'Total_Shipments', 'Pickup_Fail_%']])

print("\n--- 2. TOP 5 STORES: TRANSIT FAILURE % ---")
print(store_stats.sort_values('Transit_Fail_%', ascending=False).head(5)[['Store ID', 'City', 'Total_Shipments', 'Transit_Fail_%']])

print("\n--- 3. TOP 5 STORES: DELIVERY FAILURE % ---")
print(store_stats.sort_values('Delivery_Fail_%', ascending=False).head(5)[['Store ID', 'City', 'Total_Shipments', 'Delivery_Fail_%']])

--- 1. TOP 5 STORES: PICKUP FAILURE % ---
   Store ID       City  Total_Shipments  Pickup_Fail_%
80      S14      Delhi               37           10.8
87      S15  Hyderabad               39           10.3
2       S01      Delhi               61            9.8
11      S02       Pune               31            9.7
23      S04       Pune               32            9.4

--- 2. TOP 5 STORES: TRANSIT FAILURE % ---
   Store ID       City  Total_Shipments  Transit_Fail_%
79      S14    Chennai               48            16.7
8       S02      Delhi               55            16.4
39      S07  Hyderabad               43            16.3
17      S03       Pune               49            16.3
43      S08    Chennai               51            15.7

--- 3. TOP 5 STORES: DELIVERY FAILURE % ---
   Store ID     City  Total_Shipments  Delivery_Fail_%
29      S05     Pune               46             15.2
71      S12     Pune               42             11.9
40      S07   Mumbai               53 